In [1]:
# imports

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

### Step 1: Establish broad API call
* ensures API ok


In [2]:
import json

import requests
import os

FS_API_KEY = os.getenv('FOURSQUARE_API_KEY')


print(FS_API_KEY)

location = "Vancouver, Canada"
url = "https://api.foursquare.com/v3/places/search?near=" + location

# Create dictionary for headers

payload = {}
headers = {
        "Accept": "application/json",
        'Authorization': FS_API_KEY
            }

response = requests.request("GET", url, headers=headers, data=payload)

print(response.json())

fsq3IfwEJ3A+/5rg/MDnHEO7RgiDAxoRIW9j7zW/C0ScVlE=
{'results': [{'fsq_id': '4bdf6cddffdec9287a09eca1', 'categories': [{'id': 16039, 'name': 'Urban Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}}], 'chains': [], 'distance': 998, 'geocodes': {'main': {'latitude': 49.241325, 'longitude': -123.11111}, 'roof': {'latitude': 49.236986, 'longitude': -123.110884}}, 'link': '/v3/places/4bdf6cddffdec9287a09eca1', 'location': {'address': '4600 Cambie St', 'country': 'CA', 'formatted_address': '4600 Cambie St, Vancouver BC V5Z 2Z1', 'locality': 'Vancouver', 'postcode': 'V5Z 2Z1', 'region': 'BC'}, 'name': 'Queen Elizabeth Park', 'related_places': {'children': [{'fsq_id': '4e07cfdfb61c60b04547fd16', 'name': 'Queen Elizabeth Parking Lot'}, {'fsq_id': '4e5957dd52b1b78b819bf083', 'name': 'Celebration Pavilion'}, {'fsq_id': '637ab2e6da100869764d3264', 'name': 'Quarry Gardens'}, {'fsq_id': '4aac6667f964a520d85d20e3', 'name': 'Bloedel Floral Conserv

## Step 2: Prepare and Test Lat Long info from SQL
* ensure syntax is correct
* update Postgres DB to avoid potential concatenation complication in code.
* 

In [3]:
## SQL to create a new column in the City_bike Table. 
# Performed in Postgres

'''UPDATE city_bikes
SET lat_long = (select concat(latitude,',',longitude));'''



"UPDATE city_bikes\nSET lat_long = (select concat(latitude,',',longitude));"

## Step 3: Create DataFrame of Lat/Long and iterate to create API calls

In [38]:
import pandas as pd

data = pd.read_csv('../data/city_bikes.csv')
vancouver_stations = pd.DataFrame(data)
# vancouver_stations_TEST= vancouver_stations.head(2)
# vancouver_stations_TEST
vancouver_stations

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra_ebikes,extra_has_ebikes,extra_last_updated,lat_long
0,5,29,7a19c49f486d7c0c02b3685d7b240448,49.262487,-123.114397,10th & Cambie,2023-09-21 19:03:51.469000,6,True,1695322925,"49.262487,-123.114397"
1,12,4,32603a87cfca71d0f7dfa3513bad69d5,49.274566,-123.121817,Yaletown-Roundhouse Station,2023-09-21 19:03:51.456000,0,True,1695322945,"49.274566,-123.121817"
2,13,13,6d42fa40360f9a6b2bf641c7b8bb2862,49.279764,-123.110154,Dunsmuir & Beatty,2023-09-21 19:03:51.457000,1,True,1695322662,"49.279764,-123.110154"
3,11,5,66f873d641d448bd1572ab086665a458,49.260599,-123.113504,12th & Yukon (City Hall),2023-09-21 19:03:51.458000,4,True,1695322780,"49.260599,-123.113504"
4,9,7,485d4d24c803cfde829ab89699fed833,49.264215,-123.117772,8th & Ash,2023-09-21 19:03:51.477000,4,True,1695322949,"49.264215,-123.117772"
...,...,...,...,...,...,...,...,...,...,...,...
240,34,6,a74744ce4bb7ea2aa9f406ac8bff95d8,49.280977,-123.035969,PNE - Hastings & Windermere,2023-09-21 19:03:51.275000,2,True,1695322761,"49.280977,-123.035969"
241,8,11,5699b40126e10c2f68eefc3eb18ff3a1,49.270783,-123.141564,1st & Fir,2023-09-21 19:03:51.247000,1,True,1695322960,"49.270783,-123.141564"
242,15,7,34fd37d12eb989b49518ef53941ee3ff,49.264019,-123.209176,Sasamat & 10th,2023-09-21 19:03:51.199000,3,True,1695322610,"49.264019,-123.209176"
243,18,2,b7dd37a7dd668d6d10024b7f18acc438,49.265800,-123.205960,Trimble & 8th,2023-09-21 19:03:51.246000,1,True,1695322940,"49.2658,-123.20596"


## Foursquare API and Dataframe
### Code Overview (foursquare) 
* function takes lat and long and radius as arguments. 
* GET parameters include desired categories, and field information
* API response (json) file is iterated to retrieve data (name, popularity, price etc)
* data (dictionary) is appended to a list callled "FS_API_result"
* List of lat/longs from CityBikes (vancouver_stations) is itererated.
* Each lat/log in Vancouver_stations creates a new API call in the 'foursquare' function. 
* Results from each call (FS_API_Result) are in turn iterated and appended to the "All Results" list. 
* All_Results is a tidy dictionary that does not require json normalization. 
* DataFrame is created using All_Results dictionary. 
* DataFrame is dumped to json in ../data folder for future reference. 


In [39]:
import requests
import os
from pprint import  pprint

FS_API_KEY = os.getenv('FOURSQUARE_API_KEY')
def foursquare(latitude, longitude, radius):
    ## Initialize an empty list to store the results
    FS_API_Result = []
    
    url = 'https://api.foursquare.com/v3/places/search'
    category_filter = "11062,13003,13004,13005,13006,13007,13008,13009,13010,13011,13012,13013,13014,13015,13016,13017,13018,13019,13020,13021,13022,13023,13024,13025,13059"
    params = { "ll": f"{latitude},{longitude}", "radius": radius, "categories": {category_filter},'fields': 'name,popularity,rating,price,distance,fsq_id'}
    headers = { "Accept": "application/json", "Authorization": FS_API_KEY }
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code != 200:
        return f"Error: {response.status_code}"
    else:
        data = response.json()
        for i in data['results']:
            details = {
            'name': i.get('name', None),
            'popularity': i.get('popularity', None),
            'rating': i.get('rating', None),
            'price': i.get('price', None),
            'distance': i.get('distance', None),
            'fsq_id': i.get('fsq_id', None)
            }
            FS_API_Result.append(details)
        return FS_API_Result

all_results_FS = []  # Initialize empty list to accumulate results from each API call.
    ## Loop latitude,longitude from station to get details
for index, row in vancouver_stations.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    station_id = row['id']
    result_list = foursquare(latitude, longitude, 1000)

    for result in result_list:
        # Create a new dictionary with latitude and longitude, id of station, then update it with the result dictionary
        combined_result = {'latitude': latitude, 'longitude': longitude, 'station_id': station_id}
        combined_result.update(result)
        
        ## Append the combined dictionary to 'all_results'
        all_results_FS.append(combined_result)
pprint(all_results_FS)

[{'distance': 250,
  'fsq_id': '4f60fc8be4b0a9d090a1f850',
  'latitude': 49.262487,
  'longitude': -123.114397,
  'name': 'Rogue Kitchen & Wetbar',
  'popularity': 0.982484197700099,
  'price': 3,
  'rating': 8.2,
  'station_id': '7a19c49f486d7c0c02b3685d7b240448'},
 {'distance': 674,
  'fsq_id': '51b201d27dd249ae714ba728',
  'latitude': 49.262487,
  'longitude': -123.114397,
  'name': '33 Acres Brewing Co',
  'popularity': 0.9504036250095195,
  'price': 2,
  'rating': 9.4,
  'station_id': '7a19c49f486d7c0c02b3685d7b240448'},
 {'distance': 779,
  'fsq_id': '537e3ec8498ec730b9966b97',
  'latitude': 49.262487,
  'longitude': -123.114397,
  'name': 'The Juice Truck',
  'popularity': 0.9078516487700861,
  'price': 1,
  'rating': 8.8,
  'station_id': '7a19c49f486d7c0c02b3685d7b240448'},
 {'distance': 744,
  'fsq_id': '4cbf89f200d8370403c8445c',
  'latitude': 49.262487,
  'longitude': -123.114397,
  'name': 'Biercraft Bistro',
  'popularity': 0.9134681288553804,
  'price': 3,
  'rating': 8.1

In [34]:
pprint(all_results_FS,indent=4)

## Convert the 'all_results' list of dictionaries to a DataFrame

[   {   'distance': 250,
        'fsq_id': '4f60fc8be4b0a9d090a1f850',
        'latitude': 49.262487,
        'longitude': -123.114397,
        'name': 'Rogue Kitchen & Wetbar',
        'popularity': 0.982484197700099,
        'price': 3,
        'rating': 8.2,
        'station_id': '7a19c49f486d7c0c02b3685d7b240448'},
    {   'distance': 674,
        'fsq_id': '51b201d27dd249ae714ba728',
        'latitude': 49.262487,
        'longitude': -123.114397,
        'name': '33 Acres Brewing Co',
        'popularity': 0.9504036250095195,
        'price': 2,
        'rating': 9.4,
        'station_id': '7a19c49f486d7c0c02b3685d7b240448'},
    {   'distance': 779,
        'fsq_id': '537e3ec8498ec730b9966b97',
        'latitude': 49.262487,
        'longitude': -123.114397,
        'name': 'The Juice Truck',
        'popularity': 0.9078516487700861,
        'price': 1,
        'rating': 8.8,
        'station_id': '7a19c49f486d7c0c02b3685d7b240448'},
    {   'distance': 744,
        'fsq_id': '4

In [47]:
# pretty = json.dumps(all_results_FS.json(),indent =4)
# print(pretty)

df_foursquare = pd.DataFrame(all_results_FS)


,latitude,longitude,station_id,name,popularity,rating,price,distance,fsq_id
1,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,33 Acres Brewing Co,0.950404,9.4,2.0,674,51b201d27dd249ae714ba728
891,49.258902,-123.109596,b30ed0719dd281867a4d39779c78b58a,33 Acres Brewing Co,0.950404,9.4,2.0,631,51b201d27dd249ae714ba728
1404,49.265242,-123.095577,dcf3d5c9f1b3a46f0ef2eb2247931f89,33 Acres Brewing Co,0.950404,9.4,2.0,723,51b201d27dd249ae714ba728
30,49.260599,-123.113504,66f873d641d448bd1572ab086665a458,33 Acres Brewing Co,0.950404,9.4,2.0,694,51b201d27dd249ae714ba728
683,49.258425,-123.100792,838d5bb59963474ad08a6d5b156943ed,33 Acres Brewing Co,0.950404,9.4,2.0,687,51b201d27dd249ae714ba728
981,49.263635,-123.099236,707c1c54e0278fbae5d0ae3c42a25b71,33 Acres Brewing Co,0.950404,9.4,2.0,442,51b201d27dd249ae714ba728
410,49.262141,-123.101961,93c88a18c673e2c72a6afa93d6b89f18,33 Acres Brewing Co,0.950404,9.4,2.0,309,51b201d27dd249ae714ba728
74,49.271721,-123.104207,95e624191c655f50e401d280cd39a9ad,33 Acres Brewing Co,0.950404,9.4,2.0,877,51b201d27dd249ae714ba728
1534,49.266456,-123.100324,c2f4e05c01ea0d27934ac7dbc50cbd0c,33 Acres Brewing Co,0.950404,9.4,2.0,463,51b201d27dd249ae714ba728
1265,49.258352,-123.096337,1ef01f2f89621d51afbabaa6296e55be,33 Acres Brewing Co,0.950404,9.4,2.0,894,51b201d27dd249ae714ba728


In [69]:
# Filter DataFrame based on unique records in 'name'
df_unique_locations = df_foursquare[['name','rating','fsq_id']].drop_duplicates(subset='name', keep='first').sort_values(by='rating', ascending=False).reset_index()

# Return only the Top 10
Top10_locations_FS = df_unique_locations.head(10)

Top10_locations_FS



,index,name,rating,fsq_id
0,1,33 Acres Brewing Co,9.4,51b201d27dd249ae714ba728
1,23,Keefer Bar,9.1,4b77ba28f964a52064a82ee3
2,26,The Boxcar,9.0,55960645498e4220d6c38c64
3,416,Anh and Chi,8.9,57096a03cd10b7535b1d59a5
4,1036,Callister Brewing Co,8.9,55a177ba498e308b8bf83a86
5,20,Pourhouse Restaurant,8.9,4abea310f964a520ff8e20e3
6,82,Tap & Barrel,8.8,5205bfe411d2d4fd57c20095
7,2,The Juice Truck,8.8,537e3ec8498ec730b9966b97
8,1037,Strange Fellows Brewing,8.7,5429da79498eefacefd56aa6
9,294,JD's Barber Shop,8.7,4ad7d04df964a520000f21e3


In [42]:
## SAVE A BACKUP of MY PRECIOUS JSON FILE

import json

with open('../data/FourSquare_APIresult2_data.json', 'w') as f:
    f.write(json.dumps(json.loads(df_foursquare.to_json(orient='records')), indent=4))


In [80]:
## Create a df subset for testing when merging data later. 
df_FS_TESTING= df_foursquare.head(100)
df_FS_TESTING

,latitude,longitude,station_id,name,popularity,rating,price,distance,fsq_id
0,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Rogue Kitchen & Wetbar,0.982484,8.2,3.0,250,4f60fc8be4b0a9d090a1f850
1,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,33 Acres Brewing Co,0.950404,9.4,2.0,674,51b201d27dd249ae714ba728
2,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,The Juice Truck,0.907852,8.8,1.0,779,537e3ec8498ec730b9966b97
3,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Biercraft Bistro,0.913468,8.1,3.0,744,4cbf89f200d8370403c8445c
4,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Faculty Brewing Co,0.958609,8.3,2.0,994,55d97956498e2c299b5830c0
...,...,...,...,...,...,...,...,...,...
95,49.282409,-123.118541,6993b3dbb0758927967592ea612a2b1e,Steamworks Brewing Co,0.998420,8.5,2.0,595,4aa6bc27f964a520df4a20e3
96,49.282409,-123.118541,6993b3dbb0758927967592ea612a2b1e,Pourhouse Restaurant,0.971080,8.9,2.0,767,4abea310f964a520ff8e20e3
97,49.282409,-123.118541,6993b3dbb0758927967592ea612a2b1e,Uva Wine Bar,0.952441,7.9,2.0,337,4aa7f6f2f964a520464e20e3
98,49.282409,-123.118541,6993b3dbb0758927967592ea612a2b1e,Reflections,0.941170,7.6,3.0,162,4e4f15981f6eae4d2515db5d


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
import YELP as creds
import requests
import os
API_KEY=creds.YELP_API
# print(API_KEY) ## Confirmed - API Key ok!! 

def yelp_API_call(latitude, longitude):
    radius =1000
    Yelp_API_Result= []
    
    parameters = {
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "categories":"bars%2Crestaurants",
        "limit": 50
    }
    url = "https://api.yelp.com/v3/businesses/search"

    headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {API_KEY}" 
    }

    response = requests.get(url, params=parameters, headers=headers)
    # print(response.text)
    ###### Single API Call generated ^^
    
    #### Loop through result to extract target data and populate the Result List
    
    if response.status_code != 200:
        return f"Error: {response.status_code}"
    else:
        data = response.json()
        # print(data)
    
        for i in data['businesses']:
            details = {
            'name': i.get('name', None),
            'review_count':i.get('review_count',None),
            'rating': i.get('rating', None),
            'price': i.get('price', None),
            'distance': i.get('distance', None),
            'id': i.get('id', None)
        }
            Yelp_API_Result.append(details)
        # print(Yelp_API_Result)
        return Yelp_API_Result

#################   
   
    
all_results = []  # Initialize empty list to accumulate results from each API call.

## Loop latitude,longitude from station to get details
for index, row in vancouver_stations.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    station_id = row['id']
    result_list = yelp_API_call(latitude, longitude)
    # all_results.append(result_list)   

    
    for result in result_list:
            # Create a new dictionary with latitude and longitude, id of station, then update it with the result dictionary
            combined_result = {'latitude': latitude, 'longitude': longitude, 'station_id': station_id}
            combined_result.update(result)
            
        ## Append the combined dictionary to 'all_results'
            all_results.append(combined_result)
    print("This is all results:")
    print(all_results)    
    
    



Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [74]:
df_yelp = pd.DataFrame(all_results)
df_yelp.sort_values(by=['rating'], ascending=True)


,latitude,longitude,station_id,name,review_count,rating,price,distance,id
6088,49.261403,-123.070102,797ff1725f8d2b18337de51e240ff60d,McDonald's,18,1.5,$,46.564624,FEqXdPjcbE5tTTPWHnwtkw
2777,49.295340,-123.150433,9f63cebac9073eac3e90be2b9a91b02b,Third Beach Concession,2,1.5,$,35.062425,OXPDSeqYp9p19PDDpg-ilg
6080,49.261403,-123.070102,797ff1725f8d2b18337de51e240ff60d,Subway,12,1.5,$,30.267941,fExtjpt8trKzZC-_tFDH-w
7458,49.302577,-123.155616,45a4c865db23e1cad0209afa5a799de0,Third Beach Concession,2,1.5,$,853.655016,OXPDSeqYp9p19PDDpg-ilg
6799,49.261353,-123.085333,1c8f7fd6535db12cc45cfc59aa9da058,KFC,26,2.0,$,214.758098,8HRKVSGAHBazLZFeeVJBsQ
...,...,...,...,...,...,...,...,...,...
4067,49.258924,-123.152591,cadc004f0903ef45e898032143c0832f,NUTTEA,28,5.0,None,1026.259816,OXEaIQsgJIUCyRev5VgvLA
5828,49.265005,-123.079481,a21f40838346e300063bdda3c5c04105,Viet Family- The Vegan House,18,5.0,None,1036.233422,je53sE7RTFj9Nix0aatH3g
2487,49.291909,-123.140713,00fa94ad698dc4a9e4d708d6fd32f294,Arike Restaurant,25,5.0,None,583.945235,Q7d2jLllssfrP9u3JJGMIw
5931,49.268616,-123.069908,ebc551adb0fc1a684f4080e4954a9ba4,Giancarlo's Sports Bar,12,5.0,$,66.125390,vi9OG-2KBsAQYjybvIJvoA


Put your parsed results into a DataFrame

In [77]:
# Filter DataFrame based on unique records in 'name'
df_unique_locations = df_yelp[['name','rating','id']].drop_duplicates(subset='name', keep='first').sort_values(by='rating', ascending=False).reset_index()

# Return only the Top 10
Top10_locations_Yelp = df_unique_locations.head(10)

Top10_locations_Yelp

,index,name,rating,id
0,5380,Urban Tadka,5.0,Qw1kYX256WDxXM3zxZGsLQ
1,2592,Ssong's Hotdog,5.0,vnEmk5F8A9hzuwFVy3pBmg
2,75,Manoush'eh,5.0,K1nbiOrySlw_-XG-3NmErQ
3,6844,Golftec,5.0,7EVn1-LIXpewCZ4IY-okaQ
4,1744,NUTTEA,5.0,OXEaIQsgJIUCyRev5VgvLA
5,5361,The Garden Strathcona,5.0,viUq9OyTgOGgcdyuNoey3w
6,133,The Magnet,5.0,JGZbGEKXmZ7OeqjYaGp4bw
7,2727,Vancouver Seawall,5.0,XHJTdq8QJp6_9oCj5hU85w
8,6233,Camion Cafe,5.0,C-ddZ2lcotmzqf-grZbs1w
9,9298,BikeHike Adventures,5.0,sOejPUrDKfoTNhREFVRn_w


In [79]:
## Create a df subset for testing when merging data later. 
df_yelp_TESTING= df_yelp.head(100)
df_yelp_TESTING

,latitude,longitude,station_id,name,review_count,rating,price,distance,id
0,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,La Taqueria Pinche Taco Shop,681,4.0,$$,169.517456,6iOAgzJ0DRZNSKA3FSrrOg
1,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Saku,226,4.5,$$,178.845344,XAH2HpuUUtu7CUO26pbs4w
2,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Marulilu Cafe,286,4.0,$$,102.329817,NensKn1MSVU_rm-1Y6WlFA
3,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Uma Sushi,114,4.5,$$$,152.157897,4118Aq9LbkvUr4s719uUkA
4,49.262487,-123.114397,7a19c49f486d7c0c02b3685d7b240448,Seaport City Seafood Restaurant,30,4.5,None,168.615262,F5wxgIiZE7LYQxgqhI483A
...,...,...,...,...,...,...,...,...,...
95,49.274566,-123.121817,32603a87cfca71d0f7dfa3513bad69d5,Kaide Sushi,239,4.0,$$,380.522288,NBiBIk83cYjWnFS8QTJWyg
96,49.274566,-123.121817,32603a87cfca71d0f7dfa3513bad69d5,Le Crocodile Restaurant,441,4.5,$$$$,886.802655,GSg5v3ZJxmMPrlAXqat8tA
97,49.274566,-123.121817,32603a87cfca71d0f7dfa3513bad69d5,Fritz European Fry House,814,4.0,$,465.329891,A8ab1QjPyC10m9NAKQkliA
98,49.274566,-123.121817,32603a87cfca71d0f7dfa3513bad69d5,Incognito Coffee,181,5.0,$$,660.804144,3ByjrLF8nyArUxvO6vqJCw


# Comparing Results

Which API provided you with more complete data? Provide an explanation.
* the Yelp API data provided considerably more data both in terms of results volume and depth of information.  
* For the same inputs (245 stations). FS Yielded ~1,400 unique POI, where as Yelp provided far fewer, 


| FourSquare                                                                                                                            | YELP                                                                                                                             |
|---------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------|
| ![FourSquare coverage](/Users/jamielepard/PycharmProjects/LHL-Assessments/LHL-Statistics-Project/data/FS_initial_coverage_review.png) | ![YelpCoverage](/Users/jamielepard/PycharmProjects/LHL-Assessments/LHL-Statistics-Project/data/Yelp_initial_coverage_review.png) |


Get the top 10 restaurants according to their rating:
### Top 10
> **Foursquare** 
> ![Foursquare Top10](/Users/jamielepard/PycharmProjects/LHL-Assessments/LHL-Statistics-Project/data/Top10_Foursquare.png) 

> **Yelp** 
> ![Yelp Top10](/Users/jamielepard/PycharmProjects/LHL-Assessments/LHL-Statistics-Project/data/Top10_Yelp.png)

## Comparing API Quality (continued)

In [82]:

## Import .csv from API calls
STATIONS= pd.read_csv('../data/city_bikes.csv')
YELP = pd.read_csv('../data/Yelp_location_results.csv')
FS = pd.read_csv('../data/FS_location_results.csv')

YELP['lat_long']=YELP['latitude'].astype(str) +","+ YELP['longitude'].astype(str)
YELP['source']="yelp"
YELP_df = YELP[['name','station_id','rating','price','lat_long','distance','source','id']]
YELP_df


FS['lat_long']=FS['latitude'].astype(str) +","+ FS['longitude'].astype(str)
FS['source']="FS"
FS_df = FS[['name','station_id','rating','price','lat_long','distance','source','fsq_id']]
FS_df



,name,station_id,rating,price,lat_long,distance,source,fsq_id
0,Rogue Kitchen & Wetbar,7a19c49f486d7c0c02b3685d7b240448,8.2,3.0,"49.262487,-123.114397",250,FS,4f60fc8be4b0a9d090a1f850
1,33 Acres Brewing Co,7a19c49f486d7c0c02b3685d7b240448,9.4,2.0,"49.262487,-123.114397",674,FS,51b201d27dd249ae714ba728
2,The Juice Truck,7a19c49f486d7c0c02b3685d7b240448,8.8,1.0,"49.262487,-123.114397",779,FS,537e3ec8498ec730b9966b97
3,Biercraft Bistro,7a19c49f486d7c0c02b3685d7b240448,8.1,3.0,"49.262487,-123.114397",744,FS,4cbf89f200d8370403c8445c
4,Faculty Brewing Co,7a19c49f486d7c0c02b3685d7b240448,8.3,2.0,"49.262487,-123.114397",994,FS,55d97956498e2c299b5830c0
...,...,...,...,...,...,...,...,...
2259,The Fringe Cafe,cc25ae4f093b33ba0afd1dbc0dd20324,5.8,2.0,"49.265442,-123.187738",955,FS,4aa7f754f964a520484e20e3
2260,Coppertank Grill,cc25ae4f093b33ba0afd1dbc0dd20324,5.7,1.0,"49.265442,-123.187738",941,FS,4aa6a91df964a520864a20e3
2261,Good Co. Kits,cc25ae4f093b33ba0afd1dbc0dd20324,NaN,1.0,"49.265442,-123.187738",791,FS,6457401321f70230178ea1ea
2262,Castaway Bar and Kitchen,cc25ae4f093b33ba0afd1dbc0dd20324,NaN,2.0,"49.265442,-123.187738",809,FS,5d29827cf5c8330023780aff


In [83]:
### Combine YELp and FS to evaluate coverage

Combined_Locations = pd.concat([YELP_df,FS_df])
Combined_Locations

result=Combined_Locations.groupby(['name'])['source'].value_counts().unstack(). fillna(0).sort_index()
result.sort_values(by='yelp', ascending=False)

source,FS,yelp
name,,
Cactus Club Cafe,0.0,102.0
Saku,0.0,62.0
Banana Leaf,0.0,56.0
Hokkaido Ramen Santouka,0.0,55.0
Ramen Danbo,0.0,54.0
...,...,...
Frankie's Italian Kitchen & Bar,9.0,0.0
Frank's Barber Shop,8.0,0.0
Fox Cabaret,36.0,0.0


In [84]:
## What are the scenarios where Yelp provided more information than FS
# Calculate the count of rows that satisfy the condition
count_condition = (result['yelp'] > result['FS']).sum()

# Calculate the percentage of rows that satisfy the condition relative to the total number of rows
percent_condition = (count_condition / len(result)) * 100

# Create the Summary
output = pd.DataFrame({'Count': [count_condition], 'Percentage': [percent_condition]}, index=["Occurences"])
print(output)

            Count  Percentage
Occurences   1388   87.626263


## Conclusion: 

For the purpose of this exercise, 'coverage' is defined as the count of POI (points of interest) of predefined categories that captured by the API. Coverage was initially evaluated by gross count of the POI (ie how many restaurants were identified). Additionally, a comparative analysis was performed for each restuarant name. 

Yelp API provided considerably more coverage on Vancouver-based restaurant and bars located within a 1km radius of city-share stations. 
Consequently, data gathered from the Foursquare API is not included in further modelling efforts. 